# Problem 10: Propotional hazards

# Part I: How good is the standard error of the mean formula?

Suppose we have $N$ identically distributied normal random variables with mean $\mu_{\textrm{GT}}$ and standard deviation $\sigma_{\textrm{GT}}$.

The maximum likelihood estimate of $\mu$ is $\tilde \mu = 1/N \sum X_i$ (unsurprisingly!).

The maximum likelihood estimate of $\sigma$ is $\tilde{\sigma} = \sqrt{ 1/N \sum \left(X_i - \tilde{\mu}\right)^2}$ (unsurprisingly!). 

In Part I, we will explore the question, What is the uncertainty in $\tilde{\mu}$, the estimate of $\mu$?

The following code generates a sample of $N_{\textrm{obs}}=20$ normal random variables.

In [ ]:
NObs <- 20
original_sample <- rnorm(NObs,178.1,12.1)

## Question i
Use nonparametric bootstrap to sample $N_{\textrm{obs}}$ of the original sample. 
Compute the standard deviation of the $\tilde{\mu}$'s.

The standard deviation of $\tilde{\mu}$ is a measure of the uncertainty in the estimate of $\mu$.  

Do this for a range of $N_{\rm obs}$, ranging from 10 to 10000. 
(This is a big loop. If it is slow on your computational machine, feel free to skip, e.g., every 10th or 20th value of $N_{\rm obs}$.) . 
Plot the standard deviation of $\tilde{\mu}$ versus $N$. 

In [ ]:

# the (sqrt)variance in the estimate of the mean -- nonparemetric bootstrap

# the ground truth standard deviation of the normal distrubtion
# the standard deviation of the sample

# the (sqrt)variance in the estimate of the mean -- the textbook way, called "SEM"




## Question ii
In this simple case, according to standard textbooks, there is a formula for the uncertainty in the estimate of $\mu$, called the __standard error of the mean__, which is

$SEM = \hat{\sigma}/\sqrt{N_{\rm obs}}$.

Plot the standard deviation of $\hat{\mu}$ from non-parametric bootstrap versus $N_{\rm obs}$.

On the same axis, plot $\hat{\sigma}/\sqrt{N_{\rm obs}}$ versus $N_{\rm obs}$.

Do they agree?

In [ ]:

# plot SEM from the textbook formula, and the standard deviation of mu-hat from bootstrap to see if they agree



# Part II: Cox proportional hazards model on liver disease data


In [ ]:
# Install required packages if not already installed
if (!requireNamespace("BiocManager", quietly = TRUE)) {
  install.packages("BiocManager")
}
if (!requireNamespace("curatedBreastData", quietly = TRUE)) {
  BiocManager::install("curatedBreastData")
}
if (!requireNamespace("survival", quietly = TRUE)) {
  install.packages("survival")
}
if (!requireNamespace("ggplot2", quietly = TRUE)) {
  install.packages("ggplot2")
}

In [ ]:
# Load required libraries
library(curatedBreastData)
library(survival)
library(ggplot2)
library(dplyr)


To model an event that happens over time -- for example here, a death due to liver disease or breast cancer -- a powerful modeling framework is the Generalized Poisson Process, where the parameter is a function of time, and is called the hazard rate.
Furthermore, to predict how this depends on covariates, a commonly-used model is the Cox proportional Hazards Model.


## Question i

Learn about the Cox Proportional Hazards Model on wikipedia or elsewhere.
What is the likelihood function of the Cox Proportional Hazards Model?

In [ ]:

# likelihood function of the Cox Proportional Hazards Model


## Question ii

Access the following public dataset on breast cancer survival:

In [ ]:
# Load the curatedBreastData clinical data
data(clinicalData)

# Extract the clinical data table
clinical_df <- clinicalData$clinicalTable

# Explore survival data availability
cat("Total number of samples:", nrow(clinical_df), "\n")
cat("Samples with Overall Survival (OS) data:", sum(!is.na(clinical_df$OS)), "\n")
cat("Samples with OS months data:", sum(!is.na(clinical_df$OS_months_or_MIN_months_of_OS)), "\n")
cat("Samples with definitive follow-up until death:", sum(!is.na(clinical_df$OS_up_until_death)), "\n")

colnames(clinicalData$clinicalTable)


The following code restricts the data to a clean subset with only a few covariates for the purposes of this assignment.

In [ ]:
# Create the dataset for analysis
surv_data <- clinical_df %>%
  select(
    OS,
    OS_months_or_MIN_months_of_OS,
    HER2_preTrt,
    age,
  ) %>%
  # Filter to only include complete cases for these key variables
  filter(
    !is.na(OS),
    !is.na(OS_months_or_MIN_months_of_OS),
    !is.na(HER2_preTrt),
    !is.na(age)
  )

# Check how many complete cases we have
cat("\nNumber of complete cases for Cox model:", nrow(surv_data), "\n")

# Convert categorical variables to factors
surv_data$HER2_preTrt <- factor(surv_data$HER2_preTrt)

# Display summary statistics
cat("\nSummary of our filtered dataset:\n")
print(summary(surv_data))

How many patients are there in the cleaned dataset?

## Question iii

Use the R function to fit this data to the cox proportional hazards model on the full dataset. 
Plot the mean survival function (which is the complement of the cumulative distribution function of survival times).

In [ ]:

# Fit Cox Proportional Hazards model with the selected variables
cox_formula <- Surv(OS_months_or_MIN_months_of_OS, OS) ~ 
  HER2_preTrt + 
  age

# Initial model fit
initial_cox_model <- coxph(cox_formula, data = surv_data)
print(summary(initial_cox_model))

# Survival curve for average covariates (mean of training set)
sf_mean <- survfit(initial_cox_model)

plot(sf_mean, col = "blue", lty = 2, xlab = "Time", ylab = "Survival Probability",
     main = "Survival Curves")


## Question iv

Suppose a new patient (called `fictitious_patient`) has the following information. 

In [ ]:

fictitious_patient <- surv_data[1, , drop = FALSE]  # copy structure
fictitious_patient$HER2_preTrt <- factor("1", levels = levels(surv_data$HER2_preTrt))
fictitious_patient$age <- 45


According to the cox model you learned from the public dataset, what is their 3-year survival probability?

In [ ]:
# Survival curve for the fictitious patient
sf_fict <- survfit(initial_cox_model, newdata = fictitious_patient)

three_years_in_months <- 3*12

# Find the closest time point to 3 years in the survival curve
closest_time_idx <- which.min(abs(sf_fict$time - three_years_in_months))
three_year_surv_prob <- sf_fict$surv[closest_time_idx]

cat("\n3-year survival probability for fictitious patient:", 
    round(three_year_surv_prob * 100, 1), "%\n")

## Question v

Perform a Bootstrap by repeating the cox learning algorithm, with `n_bootstrap = 200` resamplings of the patient data.
What is the standard deviation of the estimate of the 3-year survival probability? 
Note this is assuming you have data on `nrow(surv_data)` patients.

In [ ]:

# Bootstrap with n_bootstrap = 200 and sample_size = nrow(surv_data)


## Question vi

Repeat the Bootstrap but for `sample_sizes <- 50:200`.
How many patient samples would be needed to estimate the Fictitious Patient's 3-year survival probability to within 5%?

In [ ]:
# Bootstrap for sample sizes from 50 to 200